In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Define the function for ALMA
def alma(data, window_size:int, sigma:float, offset:float):
    m = offset * (window_size - 1)
    s = window_size / sigma
    w = np.exp(-(np.arange(window_size) - m)**2 / (2 * s**2))
    w /= w.sum()
    return np.convolve(data, w, mode='valid')

In [3]:
# Load data
closing_prices = np.genfromtxt('./data/PSEI.csv', delimiter=',', skip_header=1, usecols=4)
closing_prices

array([2141.77, 2153.18, 2074.75, ..., 6923.08, 6842.79, 6876.79])

In [4]:
# Define a range of window_sizes, sigma, and offset
slow_ws = np.arange(10,20, 1)
slow_sigma = np.arange(1, 20, 1)
slow_offset = np.array([0.85, 0.9, 0.95])

fast_ws = np.arange(1,10, 1)
fast_sigma = np.arange(1, 20, 1)
fast_offset = np.array([0.85, 0.9, 0.95])

In [5]:
# Backtesting
def back_testing(slow_alma_data, fast_alma_data, data):
    # Initialize the variables
    position = 0
    entry_price = 0
    exit_price = 0
    profit = 0

    # Initialize the lists
    entry_price_list = []
    exit_price_list = []

    # Initialize the counters
    position_counter = 0
    profit_counter = 0

    # Initialize the flags
    entry_flag = False
    exit_flag = False

    # Loop through the data (slow_alma_meg, fast_alma_meg, data_meg)
    for i, j, k in zip(slow_alma_data, fast_alma_data, data):
        # If we are flat
        if position == 0:
            # If the fast ALMA crosses above the slow ALMA
            if j > i:
                # Set the entry price
                entry_price = k
                # Set the entry flag to True
                entry_flag = True
                # Set the exit flag to False
                exit_flag = False
                # Set the position to 1
                position = 1
                # Increment the position counter by 1
                position_counter += 1
        # If we are long
        elif position == 1:
            # If the fast ALMA crosses below the slow ALMA
            if j < i:
                # Set the exit price
                exit_price = k
                # Set the exit flag to True
                exit_flag = True
                # Set the entry flag to False
                entry_flag = False
                # Set the position to 0
                position = 0
                # Calculate the profit
                profit = exit_price - entry_price
                # Increment the profit counter by the profit
                profit_counter += profit
                # Append the entry price to the entry price list
                entry_price_list.append(entry_price)
                # Append the exit price to the exit price list
                exit_price_list.append(exit_price)

    return profit_counter

In [6]:
# backtesting2
# buy: mean fast alma in 10 days > mean slow alma in 10 days
# sell: mean fast alma in 10 days < mean slow alma in 10 days
# hold: no change in previous position
def back_testing2(slow_alma_data, fast_alma_data, data):
    current_position = 0 # 0 - None, 1 - Buy, -1 - Sell
    profit = 0
    
    # Divide the data into 10 days
    slow_alma_data = np.array_split(slow_alma_data, len(slow_alma_data)/10)
    fast_alma_data = np.array_split(fast_alma_data, len(fast_alma_data)/10)
    data = np.array_split(data, len(data)/10)

    for i, j, k in zip(slow_alma_data, fast_alma_data, data):
        if current_position == 0:
            if np.mean(j) > np.mean(i):
                current_position = 1
            elif np.mean(j) < np.mean(i):
                current_position = -1
        elif current_position == 1:
            if np.mean(j) < np.mean(i):
                current_position = 0
                profit += k[-1] - k[0]
        elif current_position == -1:
            if np.mean(j) > np.mean(i):
                current_position = 0
                profit += k[0] - k[-1]
    
    return profit

In [7]:
# Test each combination of window_size, sigma, and offset and find the best combination that yields the highest profit (use the back_testing function)

# Initialize the variables
best_profit = None
best_window_size = None
best_sigma = None
best_offset = None

# Loop through the slow_ws
for sws in slow_ws:
    # Loop through the slow_sigma
    for ss in slow_sigma:
        # Loop through the slow_offset
        for so in slow_offset:
            # Loop through the fast_ws
            for fws in fast_ws:
                # Loop through the fast_sigma
                for fs in fast_sigma:
                    # Loop through the fast_offset
                    for fo in fast_offset:
                        # Calculate the slow ALMA
                        slow_alma = alma(closing_prices, sws, ss, so)
                        # Calculate the fast ALMA
                        fast_alma = alma(closing_prices, fws, fs, fo)
                        # Calculate the profit
                        profit = back_testing(slow_alma, fast_alma, closing_prices)
                        # If the profit is greater than the best profit
                        if best_profit is None or profit > best_profit:
                            # Set the best profit
                            best_profit = profit
                            # Set the best window_size
                            best_window_size = (sws, fws)
                            # Set the best sigma
                            best_sigma = (ss, fs)
                            # Set the best offset
                            best_offset = (so, fo)

# Print the best profit, best window_size, best sigma, and best offset
print("Based on entry and exit signals")
print(f'Best Profit: {best_profit}\nBest Window Size: {best_window_size}, Best Sigma: {best_sigma}, Best Offset: {best_offset}')

Based on entry and exit signals
Best Profit: 113966.85000000006
Best Window Size: (10, 6), Best Sigma: (17, 13), Best Offset: (0.95, 0.85)


In [8]:
# Test each combination of window_size, sigma, and offset and find the best combination that yields the highest profit (use the back_testing function)

# Initialize the variables
best_profit = None
best_window_size = None
best_sigma = None
best_offset = None

# Loop through the slow_ws
for sws in slow_ws:
    # Loop through the slow_sigma
    for ss in slow_sigma:
        # Loop through the slow_offset
        for so in slow_offset:
            # Loop through the fast_ws
            for fws in fast_ws:
                # Loop through the fast_sigma
                for fs in fast_sigma:
                    # Loop through the fast_offset
                    for fo in fast_offset:
                        # Calculate the slow ALMA
                        slow_alma = alma(closing_prices, sws, ss, so)
                        # Calculate the fast ALMA
                        fast_alma = alma(closing_prices, fws, fs, fo)
                        # Calculate the profit
                        profit = back_testing2(slow_alma, fast_alma, closing_prices)
                        # If the profit is greater than the best profit
                        if best_profit is None or profit > best_profit:
                            # Set the best profit
                            best_profit = profit
                            # Set the best window_size
                            best_window_size = (sws, fws)
                            # Set the best sigma
                            best_sigma = (ss, fs)
                            # Set the best offset
                            best_offset = (so, fo)

# Print the best profit, best window_size, best sigma, and best offset
print("Based on 2 week mean")
print(f'Best Profit: {best_profit}\nBest Window Size: {best_window_size}, Best Sigma: {best_sigma}, Best Offset: {best_offset}')

Based on 2 week mean
Best Profit: 26212.03
Best Window Size: (18, 9), Best Sigma: (12, 1), Best Offset: (0.85, 0.85)
